### **Домашнее задание к лекции "Основы веб-скрапинга"**

**Обязательная часть**

Вам необходимо написать функцию, которая будет основана на поиске по сайту habr.com. Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

<дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.

**Дополнительная часть (необязательная)**

Функция из обязательной части задания должна быть расширена следующим образом:

кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

Необходимые импорты

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime

pd.options.display.max_colwidth = None

Список ключевых слов для запросов

In [6]:
keyword_list = ['python', 'анализ данных', 'deepseek', 'пенза'] 

In [ ]:
def get_queries(query_list: list):

    """Функция для формирования датафрейма(ов) на основании переданного
    списка ключевых слов. 

    Returns:
        list: список датафреймов, сформированных на основании элементов переданного
            в качестве параметра списка.
    """

    url = 'https://habr.com/ru/search/'
    query_result_list = []

    # пройдемся по переданному списку ключевых слов
    for query in query_list:
        rows = []
        # передадим в параметры запроса ключевое слово
        params = {
            'q': query,
            'target_types': 'posts',
            'order': 'date'
        }

        # выполним запрос и создадим объект BeautifulSoup на основании полученного
        # ответа для дальнейшей обработки
        res = requests.get(url, params)
        time.sleep(0.2)
        soup = BeautifulSoup(res.text)

        # заберем все ссылки на первой странице выдачи в список и сделаем их 
        # полными
        refs_list = soup.find_all('a', class_='tm-title__link')

        links_list = []
        all_links = []

        for ref in refs_list:
            links_list.append(ref.get('href'))

        full_link = list(map(lambda x: 'https://habr.com' + x, links_list))

        all_links += full_link

        # используя полученный список ссылок, сделаем по ним запросы, чтобы
        # извлечь требуемые данные - дату, заголовок и саму ссылку
        for link in all_links:
            res = requests.get(link)
            time.sleep(0.2)
            soup = BeautifulSoup(res.text)
            res_datetime = soup.find(
                'span', class_='tm-article-datetime-published').find('time')['datetime']
            
            res_datetime = datetime.fromisoformat(res_datetime.replace('Z', '+00:00'))
            res_datetime = res_datetime.strftime("%d.%m.%Y")

            title = soup.find('h1', class_='tm-title tm-title_h1').find('span').text
            row = {'date':res_datetime,
                   'title': title,
                   'link': link}
            rows.append(row)

        # на основании полученного списка сформируем датафрейм и добавим его
        # в возвращаемый результат работы функции
        query_df = pd.DataFrame(rows).reset_index(drop=True)
        query_result_list.append(query_df)

    # поработаем над удалением дублей новостей, встречающихся в разных запросах

    # Колонка, по которой проверяем дубликаты
    column_to_check = 'link'

    # Собираем все значения из всех датафреймов
    all_values = pd.concat([df[column_to_check] for df in query_result_list])

    # Находим дубликаты
    duplicates = all_values[all_values.duplicated(keep=False)].unique()

    # Удаляем дубликаты из всех датафреймов, кроме первого
    for i, df in enumerate(query_result_list):
        if i > 0:  # Пропускаем первый датафрейм
            query_result_list[i] = df[~df[column_to_check].isin(duplicates)]

    return query_result_list

In [7]:
for df in get_queries(keyword_list):
    display(df)

,date,title,link
0,13.02.2025,Обучить модель RoBERTa расстановке запятых на балконе для продакшена,https://habr.com/ru/articles/882276/
1,13.02.2025,Биперная музыка на Arduino,https://habr.com/ru/companies/ruvds/articles/880206/
2,13.02.2025,От старого железа до первого VDS: как я собрал хостинг в студенческом общежитии,https://habr.com/ru/articles/882210/
3,13.02.2025,Шпаргалка по типам Julia для инженеров и не только,https://habr.com/ru/companies/etmc_exponenta/articles/882178/
4,13.02.2025,Книга: «Blue Fox: взлом и реверс-инжиниринг ARM»,https://habr.com/ru/companies/piter/articles/881512/
5,13.02.2025,Исследование и восстановление блока SRS Audi A4: особенности работы с закрытым процессором и glitch-технологией,https://habr.com/ru/articles/879304/
6,13.02.2025,В Минцифры работают талантливые сотрудники,https://habr.com/ru/articles/882130/
7,13.02.2025,Утверждены параметры эксперимента по внедрению системы добровольного подтверждения компетенций для разработчиков ПО в РФ,https://habr.com/ru/news/882096/
8,13.02.2025,SAST приложений под Android: делаем код безопаснее,https://habr.com/ru/companies/otus/articles/881110/
9,13.02.2025,Соревнование VN1: чему я научился у прогнозистов,https://habr.com/ru/companies/bothub/articles/881990/


,date,title,link
0,13.02.2025,"Что делать, если ты первый AppSec-инженер в компании? План работ на стартовые полгода",https://habr.com/ru/articles/882198/
1,13.02.2025,"Как «Наруто», «Тетрадь смерти», «Атака титанов» и другие аниме учат нас грамотно управлять аутстафф-проектами",https://habr.com/ru/companies/agima/articles/882140/
2,13.02.2025,ChatGPT решает гробы с экзаменов в ШАД,https://habr.com/ru/articles/881858/
3,13.02.2025,От счетных палочек до ПАКов,https://habr.com/ru/companies/timeweb/articles/882268/
4,13.02.2025,«Ростех» разработал систему для противодействия программам‑шифровальщикам,https://habr.com/ru/news/882262/
5,13.02.2025,EKF Impulse: новое имя в зарядке электротранспорта,https://habr.com/ru/companies/habr_ekf/news/882238/
6,13.02.2025,Выявляем аномалии с помощью Isolation Forest,https://habr.com/ru/companies/otus/articles/881086/
7,13.02.2025,Экосистема для разработки и применения Computer Vision (CV) в промышленности,https://habr.com/ru/articles/882204/
8,13.02.2025,Трекинг медийной рекламы: как повысить эффективность медийных размещений и не тратить бюджет впустую,https://habr.com/ru/articles/882200/
9,13.02.2025,Пять болей аналитиков и как с ними справляться,https://habr.com/ru/companies/korus_consulting/articles/882146/


,date,title,link
0,13.02.2025,Как ИИ войти в тестирование: методики разработки автотестов,https://habr.com/ru/companies/rshb/articles/881072/
1,13.02.2025,Феномен DeepSeek: разбираем причины шума вокруг нейросети,https://habr.com/ru/articles/882162/
2,13.02.2025,OpenAI отменяет выпуск своей AI-модели o3 в пользу «унифицированного» релиза нового поколения,https://habr.com/ru/companies/bothub/news/882218/
3,13.02.2025,Генеральный директор Anthropic Дарио Амодей предупреждает о «гонке» за пониманием AI по мере его усиления,https://habr.com/ru/companies/bothub/news/882180/
4,13.02.2025,"Железное будущее, Мона Лиза, марсоход: пишем на Хабр, побеждаем в Технотексте-7",https://habr.com/ru/companies/habr/articles/882172/
5,13.02.2025,Проблемы языковых моделей при анализе длинных текстов: выводы исследования,https://habr.com/ru/companies/bothub/news/882166/
6,12.02.2025,"DeepSeek, ChatGPT, YandexGPT, GigaChat и практическая школьная задачка по физике. Можно ли им доверять?",https://habr.com/ru/articles/881910/
8,12.02.2025,Лучшее за неделю (03.02 — 09.02),https://habr.com/ru/companies/ruvds/news/881680/
9,12.02.2025,Разработка скрипта для обхода Geetest CAPTCHA на Python: от идеи до реализации,https://habr.com/ru/articles/881674/
10,12.02.2025,Как сделать чат-бот с RAG безопаснее?,https://habr.com/ru/companies/raft/articles/881350/


,date,title,link
0,08.01.2025,Оцифровываем музыку из XIX века,https://habr.com/ru/articles/872176/
1,29.10.2024,"Смолатон — вот это действительно был марафон на 22 часа кодинга, а еще презентация, MVP и защита проекта",https://habr.com/ru/articles/854216/
2,09.10.2024,Разложение (проституирование) инженерной деятельности как ключевая причина краха советской системы,https://habr.com/ru/articles/849116/
3,14.09.2024,Настройка GoodByeDPI для разных регионов и провайдеров,https://habr.com/ru/articles/843272/
4,16.07.2024,Радиоуправляемые трагги как семейное увлечение: три наших «питомца»,https://habr.com/ru/companies/lanit/articles/825780/
5,30.05.2024,"Результаты большого техписательского опроса. География, демография, зарплаты",https://habr.com/ru/companies/documentat/articles/818199/
6,30.05.2024,"Дух романтики «полевых» инженеров: серверы, вулканы и медведи",https://habr.com/ru/companies/croc/articles/818303/
7,18.03.2024,Советские и постсоветские НИИ и КБ как конвейер уничтожения ресурсов развития,https://habr.com/ru/articles/801001/
8,15.03.2024,Курс «Континент 4 Getting Started 2.0»,https://habr.com/ru/companies/tssolution/articles/800291/
9,05.03.2024,Рога переходного периода (из ниоткуда в никуда),https://habr.com/ru/articles/798101/


In [2]:
def get_queries_advanced(query_list: list, num_pages=1):

    """Функция для формирования датафрейма(ов) на основании переданного
    списка ключевых слов. 

    Returns:
        list: список датафреймов, сформированных на основании элементов переданного
            в качестве параметра списка.
    """

    
    query_result_list = []

    # пройдемся по переданному списку ключевых слов
    for query in query_list:
        # передадим в параметры запроса ключевое слово
        params = {
                'q': query,
                'target_types': 'posts',
                'order': 'date'
            }
        
        rows = []

        for num in range(1, num_pages+1):
            url = f'https://habr.com/ru/search/page{num}/'
                        
            # выполним запрос и создадим объект BeautifulSoup на основании полученного
            # ответа для дальнейшей обработки
            res = requests.get(url, params)
            time.sleep(0.2)
            soup = BeautifulSoup(res.text)

            # заберем все ссылки на первой странице выдачи в список и сделаем их 
            # полными
            refs_list = soup.find_all('a', class_='tm-title__link')

            links_list = []
            all_links = []

            for ref in refs_list:
                links_list.append(ref.get('href'))

            full_link = list(map(lambda x: 'https://habr.com' + x, links_list))

            all_links += full_link

            # используя полученный список ссылок, сделаем по ним запросы, чтобы
            # извлечь требуемые данные - дату, заголовок и саму ссылку
            for link in all_links:
                res = requests.get(link)
                time.sleep(0.2)
                soup = BeautifulSoup(res.text)
                res_datetime = soup.find(
                    'span', class_='tm-article-datetime-published').find('time')['datetime']
                
                res_datetime = datetime.fromisoformat(res_datetime.replace('Z', '+00:00'))
                res_datetime = res_datetime.strftime("%d.%m.%Y")

                title = soup.find('h1', class_='tm-title tm-title_h1').find('span').text
                row = {'date':res_datetime,
                    'title': title,
                    'link': link}
                rows.append(row)
            
        # на основании полученного списка сформируем датафрейм и добавим его
        # в возвращаемый результат работы функции
        query_df = pd.DataFrame(rows).reset_index(drop=True)
        query_result_list.append(query_df)

    # поработаем над удалением дублей новостей, встречающихся в разных запросах

    # Колонка, по которой проверяем дубликаты
    column_to_check = 'link'

    # Собираем все значения из всех датафреймов
    all_values = pd.concat([df[column_to_check] for df in query_result_list])

    # Находим дубликаты
    duplicates = all_values[all_values.duplicated(keep=False)].unique()

    # Удаляем дубликаты из всех датафреймов, кроме первого
    for i, df in enumerate(query_result_list):
        if i > 0:  # Пропускаем первый датафрейм
            query_result_list[i] = df[~df[column_to_check].isin(duplicates)]

    return query_result_list

In [3]:
keyword_list = ['python'] 

for df in get_queries_advanced(keyword_list, 2):
    display(df)

,date,title,link
0,13.02.2025,Обучить модель RoBERTa расстановке запятых на балконе для продакшена,https://habr.com/ru/articles/882276/
1,13.02.2025,Биперная музыка на Arduino,https://habr.com/ru/companies/ruvds/articles/880206/
2,13.02.2025,От старого железа до первого VDS: как я собрал хостинг в студенческом общежитии,https://habr.com/ru/articles/882210/
3,13.02.2025,Шпаргалка по типам Julia для инженеров и не только,https://habr.com/ru/companies/etmc_exponenta/articles/882178/
4,13.02.2025,Книга: «Blue Fox: взлом и реверс-инжиниринг ARM»,https://habr.com/ru/companies/piter/articles/881512/
5,13.02.2025,Исследование и восстановление блока SRS Audi A4: особенности работы с закрытым процессором и glitch-технологией,https://habr.com/ru/articles/879304/
6,13.02.2025,В Минцифры работают талантливые сотрудники,https://habr.com/ru/articles/882130/
7,13.02.2025,Утверждены параметры эксперимента по внедрению системы добровольного подтверждения компетенций для разработчиков ПО в РФ,https://habr.com/ru/news/882096/
8,13.02.2025,SAST приложений под Android: делаем код безопаснее,https://habr.com/ru/companies/otus/articles/881110/
9,13.02.2025,Соревнование VN1: чему я научился у прогнозистов,https://habr.com/ru/companies/bothub/articles/881990/


In [ ]:
# query_result_list.append(soup)

        # top_level_tag = soup.find('div', class_='tm-pagination__pages')

        # nested_tags = top_level_tag.find_all(
        #     'div', class_='tm-pagination__page-group')
        # last_nested_tag = nested_tags[-1] if nested_tags else None
        # last_page = last_nested_tag.find_all(
        #     'a', class_='tm-pagination__page')[-1].text

        # print(last_page)

# def get_query_result(query):
#     article_list = query.find_all('article', class_='tm-articles-list__item')
#     return article_list

# query_list = get_queries(url, keyword_list)